In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


##### Initialization Code

In [8]:
system"l init.q"

//subsequent calls to this init will throw an error because we have changed directory 
    //this error can be safely ignored 

//if you need to do a hard reset, please restart the kernel. 

**Learning objectives**

* What are attributes ?
* How to identify an attribute?
* How to apply an attribute ?
* Performance


# Attributes

Lists (and by extension, dictionaries and columns of tables) can have attributes applied to them. Attributes are simply a way of tagging your data so that kdb+/q knows how it is arranged/ordered before attempting any lookup!

**Some features of attributes:**

* Attributes imply or enforce certain properties on the list which will aid searching in different situations.
* A list can only have one attribute set - applying another attribute to a list will overwrite the previous one.
* Some attributes will also be lost upon modification of the list.
* You will see improved performance only on lists with more than a million items.
 

Below is a description of the four [attributes](https://code.kx.com/q4m3/8_Tables/#88-attributes) used in kdb+/q to improve efficiency.

| Attribute | Purpose | Requirement | Space |Example - good | Example - fail|
|-----------|--------|------------|---------|-----------|---------|
|Sorted | Allows q to perform a binary search | Must be in ascending order | None| ```s#1 2 3 4 5``|```s#5 2 3 4 1``| 
|Unique | Allows q to perform a hashing algorithm to find the exact index for the element | Must be unique |Large|`u#2 1 3 5 4| `u#2 1 3 5 5| 
|Parted |It generates a dictionary mapping each element in the list to its first occurrence. | Identical items are stored contiguously | Small |```p#2 2 3 3 5`` | ```p#2 2 3 5 3``|
|Grouped | An internal dictionary is built and maintained which maps each unique item to each of its indices of occurrence | None | Largest| `g#2 2 3 3 5|

The application of attributes to kdb+/q data is very important to ensure best Query and Join performance. 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:12px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Setting or unsetting attributes other than sorted causes a copy of the object being created. </i></p>

## Identifying Attributes

Attribute flags are descriptive, not prescriptive; amends and appends preserve flags if the attribute is preserved. In other words, a sorted list remains sorted until an element is inserted in such a manner that disrupts the sort. Furthermore, applying an attribute to a list will result in a fail if the list does not fulfill the attribute's requirements. 

If we want to check if an attribute is applied to a list we can check use the [`attr`](https://code.kx.com/q/ref/attr/) keyword: 

In [9]:
attr 1 2 3 4
attr asc 1 2 3 4   //ascending a list automatically applies the sorted attribute


s


We can also see by the console output is an attribute is applied to a list, as it will be prefaced by <code>\`s#,\`u#,\`p#,\`g#</code>, depending on the applied attribute. 

In [10]:
1 2 3 4
asc 1 2 3 4 

1 2 3 4
`s#1 2 3 4


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Note, the match operator <code>~</code> doesn't check for applied attributes! </i></p>

In [7]:
asc[1 2 3 4]~1 2 3 4

1b


## Applying attributes 

Attributes can be explicitly applied by specifying <code>\`s#,\`u#,\`p#,\`g#</code> before the list: 

In [11]:
`s#1 2 3 4 
attr asc 10 20 30 40

`s#1 2 3 4
s


For sorting, the `asc` and `xasc` keywords will sort the data and then automatically apply the attribute: 

In [12]:
asc 1 2 3 4

`s#1 2 3 4


We can also apply attributes using:

* [@](https://code.kx.com/q/ref/amend/) - The amend at function 
* [update](https://code.kx.com/q/ref/update/) - The update function 

In [13]:
show l:1 3 5 7 9 11
@[`s#;l];  //we can use the amend function if we want to apply an attribute to a list
l

`s#1 3 5 7 9 11
1 3 5 7 9 11


Using the update function, we can apply an attribute to a column in a table:

In [14]:
t:([]time:07:00 08:00 09:00; name: `John`Ann`Mary;activity:`breakfast`walk`dinner) //defining table
meta t
update `s#time from `t //updating time column to have the sorted attribute
meta t

c       | t f a
--------| -----
time    | u    
name    | s    
activity| s    
t
c       | t f a
--------| -----
time    | u   s
name    | s    
activity| s    


All other attributes need to be explicitly applied. Note, you can only have one attribute applied to a list at any given time: 

In [15]:
`g#`s# 1 2 3 4

`g#1 2 3 4


If an attribute application fails, you'll receive an `evaluation error: attribute-fail` message - try not to take it personally!

In [ ]:
`u#1 2 2 3 

We can remove attributes by using ```#``

In [18]:
`#l

1 3 5 7 9 11


## Sorted
A preceding <code>\`s#</code> indicates that a list is sorted in ascending order. If a list is explicitly sorted by [asc](https://code.kx.com/q/ref/asc/) (or [xasc](https://code.kx.com/q/ref/asc/#xasc) for tables) then the list will automatically have the sorted attribute set to it.

In [19]:
example:([]a:1 2 3;b: `mini`example`table)
meta `a xasc example

c| t f a
-| -----
a| j   s
b| s    


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:12px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> The sorted attribute is used in Real time database (RDB) processes in kdb+/q. This process stores data that is received intraday in in-memory tables - since messages are arriving in time order, the sorted attribute is usually applied to the time column to improve query performance. </i></p>

If the list that we would like to apply the sorting attribute to **is not** sorted, we will receive a ``evaluation error: s-fail`` error: 

In [ ]:
`s#4 5 1 //will throw an error

We can also apply the sorted attribute to dictionaries and keyed tables:

In [20]:
dict:`s#10 20 30 40 50!`apple`pear`orange`apple`orange //creating a dictionary
kt: `a xkey example                                    //creating a keyed table
meta `s#kt                                             //it applies the attribute to the keyed column

c| t f a
-| -----
a| j   s
b| s    


Using a list with the sorted attribute applied to, kdb+/q can perform [binary search](https://en.wikipedia.org/wiki/Binary_search_algorithm) to find elements rather than scanning the entire list. Let's look at what binary search is:

<img src="../images/binarySearch.png" width="400" height="400">

In [35]:
N:10000000
sL:`s#til N   //generating our sorted list and applying the attribute
L:1_3,sL  
show d:3,sL
show e:1_d //removing the sorted attribute by joining a bigger value at the start, then removing

3 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 2..
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 ..


We can now see that we have two lists, only one of which has the attribute applied - other than that the lists are exactly the same: 

In [22]:
//reducing console size 
\c 20 20   
L
sL
//changing console size back to default
\c 25 80 
L~sL

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 ..
`s#0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 ..
1b


To demonstrate the performance improvement we can time some function executions:

In [36]:
\t:100 L within (20; 2391002)
\t:100 (N-1) in L

786
655


In [37]:
\t:100 sL within (20; 2391002)
\t:100 (N-1) in sL

483
0


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:2px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> When a list has a sorted attributed applied to it, linear search is replaced with binary search which is a faster searching mechanism. </i></p>

 ##### Preservation of attribute

The sorted attribute is only preserved if we supply input that complies with the attribute and when we use the modify-in-place assignment: 

In [38]:
sL,N+1   //sorted attribute lost

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 ..


In [39]:
sL,:N+1  //sorted attribute retained
sL

`s#0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 ..


However, because of it's importance to query performance in tables it is preserved on table `insert` and `upsert`: 

In [40]:
.tbl.nsExample:([]a:1 2 3;c:(1 2;3 4;5 6);d: `sneaky`namespace`table)
show sExample:`a xasc .tbl.nsExample      //sorting by column `a
meta sExample                             //sorted attribute applied
`sExample insert (4;7 8;`new)             //append by reference using insert 
                                          //(if you're getting an insert error, run all cells above)
meta sExample                             //preserved 
sExample

c| t f a
-| -----
a| j   s
c| J    
d| s    
,3
c| t f a
-| -----
a| j   s
c| J    
d| s    
a c   d        
---------------
1 1 2 sneaky   
2 3 4 namespace
3 5 6 table    
4 7 8 new      
a c   d        
---------------
1 1 2 sneaky   
2 3 4 namespace
3 5 6 table    


In [41]:
sExample:`a xasc example              //sorting by column `a              
meta sExample                         //sorted attribute applied
show sExample upsert (4;`new)         //appending by value - sExample hasn't been modified, new table returned
meta sExample upsert (4;`new)         //new table has sorted attribute applied

c| t f a
-| -----
a| j   s
b| s    
c| t f a
-| -----
a| j   s
b| s    
a b      
---------
1 mini   
2 example
3 table  
4 new    


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Only the sorted attribute is preserved when appending to disk. </i></p>

## Unique
\`u# can be applied to a list of unique elements. When a list is flagged as unique, an internal hash map is created to each item in the list which uses storage and adds overhead. By applying the ``\`u`` to a list, it allows kdb+/q to use a hashing algorithm to find the exact index for the element we are searching.       

In [42]:
`u#10 20 30 45 

`u#10 20 30 45


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> No linear searches are performed when we apply the unique attribute. </i></p>

If the list is not unique, a `evaluation error: u-fail` error will occur:

In [ ]:
`u#10 20 10 30 45 //this will throw an error because the list is not unique

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> The unique attribute is usually applied to keyed tables and ensures what we lookup from these tables is consistent. In many real time subscriber (RTS) processes, tables are created which store the last known price of a trade, or meter reading etc. These tables are typically keyed by the stock or sensor etc. and are good candidates for the unique attribute. </i></p>

In [47]:
example
show example[`b]: `u#example[`b]
`b xkey example
meta `b xkey example

a b      
---------
1 mini   
2 example
3 table  
b      | a
-------| -
mini   | 1
example| 2
table  | 3
c| t f a
-| -----
b| s   u
a| j   s
`u#`mini`example`table


We can test the performance ourselves: 

In [49]:
// creating a list
N:10000000
show L:neg[N]?`6   //creates a symbol with random letters
show uL:`u#L

`dckamm`eciokj`klpboe`fejhpp`kfjpfc`cdajfa`bjakdg`abffek`apedol`eopamb`cafoho..
`u#`dckamm`eciokj`klpboe`fejhpp`kfjpfc`cdajfa`bjakdg`abffek`apedol`eopamb`caf..


Let's try finding the last symbol in both lists:

In [50]:
\t:100 L?last L
\t:100 uL?last L

724
0


##### Preservation of attribute
An amend that does not preserve uniqueness causes the attribute to be lost:

In [51]:
uL       
uL,:`3   //amending the list doesn't preserve uniqueness
uL

`u#`dckamm`eciokj`klpboe`fejhpp`kfjpfc`cdajfa`bjakdg`abffek`apedol`eopamb`caf..
`u#`dckamm`eciokj`klpboe`fejhpp`kfjpfc`cdajfa`bjakdg`abffek`apedol`eopamb`caf..


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If you have a column that will always have unique values and your large table does not change often, you can get a significant performance speedup with `u#. It's important to remember that applying the unique attribute will consume memory as we use the hash table to perform lookups.  </i></p>

## Parted
The parted attribute \`p# means identical items are stored contiguously in memory, or on disk. The attribute can only be set by explicitly applying it. This will internally generate a dictionary mapping each element in the list to its first occurrence.

Here is a simple example of a parted list:

In [52]:
`p#1 1 1 1 3 3 3 2 2 2 2 //list is not sorted or unique

`p#1 1 1 1 3 3 3 2 2 2 2


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> In kdb+/q processes which loading in on-disk data called Historical Databases (HDBs), the symbol column will almost always have the parted attribute associated with it. This is because the symbol column is the most used query filter after date and so can benefit the most from improved search efficiency. </i></p>

In practice, to apply the parted attribute we need to structure our data into contiguous chunks corresponding to blocks of the same value e.g. <code>\`AAPL\`AAPL\`BAC`BAC`BAC...</code> and the easiest way to actually achieve that is via sorting: 

In [53]:
`p#asc upper 10?`1  //generting 10 symbols with one character

`p#`A`A`B`D`D`F`G`G`L`N


The parted attribute is **applied almost exclusively to historical data** - this is because with real-time data we don't control the grouping of the data that's arriving and the benefit of the attribute doesn't exceed the overhead that would be required to keep it in a parted format (resorting on each new data arrival). 

What happens if we have a list that doesn't take the format above?

In [ ]:
`p#1 1 1 1 3 3 3 2 2 2 2 1 // this will throw a u-fail error

Applying a sorted attribute to a table applies the parted attribute to the first column:

In [54]:
show meta example:([]a:1 2 3;b: `mini`example`table)
meta `s#example

c| t f a
-| -----
a| j   p
b| s    
c| t f a
-| -----
a| j    
b| s    


To demonstrate the performance improvement with the parted attribute, let's again time some function. Firstly let's create a list that we will apply a function to it:

In [55]:
.Q.A    //returns alphabet in CAPS
L:5 rotate .Q.A //using rotate to shuffle list 
`$L
`$'L // creating a list of symbols

ABCDEFGHIJKLMNOPQRSTUVWXYZ
FGHIJKLMNOPQRSTUVWXYZABCDE
`F`G`H`I`J`K`L`M`N`O`P`Q`R`S`T`U`V`W`X`Y`Z`A`B`C`D`E


Now we are going to create a list that contains 400,000 occurrences of each letter: 

In [56]:
L:`$'L

400000#'L

L:raze 400000#'L //flatten the matrix 
show L
count L

`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`F`..
F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F..
G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G G..
H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H H..
I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I I..
J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J J..
K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K K..
L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L L..
M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M..
N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N..
O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O..
P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P P..
Q Q Q Q Q Q Q Q Q Q Q Q Q Q Q Q Q Q Q Q 

Using the list above, let's create a parted list and check to see if we see any performance improvements:

In [57]:
pL:`p#L

\t:100 (last L) in L
\t:100 (last L) in pL

654
0


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Using the parted attribute tables do not have to be scanned linearly! </i></p>

##### Preservation of attribute

The parted attribute is not preserved under any operation on the list, even if the operation preserves the property with one exception: 
 

In [58]:
L:`p#1 1 1 3 3   //creating a parted list
L,:3             //joining 3 to the end of the list
L                //parted attribute isn't reserved

1 1 1 3 3 3


If you try to join 2 parted lists, the attribute is preserved

In [59]:
L:`p#1 1 1 3 3 //redefining L
L1:`p#3 3 3 4 4 
L,L1

`p#1 1 1 3 3 3 3 3 4 4


## Grouped
The grouped attribute is similar to the parted attribute in the sense that it keeps track of the location of the data. One of the main differences is that the grouped attribute stores a map of **ALL** occurrences compared to just storing the first occurrence of each identical item.  Applying a \`g# attribute means that the list is grouped. 

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> The grouped attribute is used with both real-time and on-disk historical tables in kdb+/q. It is often applied in real-time databases (RDBs) to the stock or sensor column to enable faster query time intraday (in conjunction with the sorted attribute on time we discussed previously). When used with on-disk historical data, this is typically applied to other highly queried secondary filter columns e.g. Broker, Exchange, Location, Machine etc </i></p>

When a grouped list is searched, the indices of the element(s) to be found are retrieved from the internal mapping. All occurrences of the element can then be retrieved.  This greatly decreases retrieval time when data is stored in-memory.

In [60]:
show L:`g#10?10

`g#9 3 3 7 5 7 5 6 8 8


While we can't access the internal map when the grouped attribute is applied, it works the same way as the `group` function

In [61]:
group "mississippi"

m| ,0
i| 1 4 7 10
s| 2 3 5 6
p| 8 9


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i>Like unique and parted, linear search is not used when we apply the grouped attribute to a list!</i></p>

This attribute can be applied to any list, as it doesn't assume anything about the lists structure.
There is an overhead associated with the internal mapping so this is normally reserved for large lists.

<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:22px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If the data can be sorted such that the parted attribute can be set, you can see a better performance compared to the grouped attribute, both on disk and in memory.</i></p>

# Performance

When we are looking up a table in kdb+/q, we should be aware that we are performing a linear search on the first column to find all the matches. From what we discussed above, we know that we should apply an attribute to the first column and therefore it will perform the lookup much faster.

## Define table and test without attributes
Let's create a table and test this theory:

In [62]:
n:100000
t:([sym:n?`AAPL`MSFT`GS;date:n?.z.d+til 20];price:10+n?10.) //creating a table containing 100,000 records
meta t

c    | t f a
-----| -----
sym  | s    
date | d    
price| f    


In [63]:
\t:100 t`date`sym!(2021.02.20;`AAPL)  //lookup without attribute

57


In [64]:
`sym`date xasc `t                //applying the sorted attribute, which gets applied to the first column in the list
meta t
\t:100 t`date`sym!(2021.02.20;`AAPL) 

t
c    | t f a
-----| -----
sym  | s   s
date | d    
price| f    
70


Let's look at how these attributes will speed up qSql queries:

In [65]:
n:1000000
t:([sym:n?`AAPL`MSFT`GS;date:n?.z.d+til 20];price:10+n?10.) //creating a bigger table 

In this small experiment, we will consider three different tables:
* table that has no attributes 
* table that has a sorted attribute on date 
* table that has a grouped attribute on sym 

### Querying a table with no attributes 
#### Query 1: No attributes
We will keep the queries constant in all three scenarios and therefore we can compare:

In [66]:
//query 1
//table with no attribute
\t:1000 res:select count i from t where date=2021.03.07
res

907
x
-
0


#### Query 2: No attributes

In [67]:
//query 2
//table with no attribute
\t:1000 res:select avg price from t where sym=`AAPL
res

2413
price  
-------
15.0067


### Querying a table with sorted attributes
Let's apply the sorted attribute to the `date` column and run both queries again:

In [68]:
//table with sorted attribute
t:`date xasc t
meta t

c    | t f a
-----| -----
sym  | s    
date | d   s
price| f    


#### Query 1: Sorted attribute

In [69]:
//query 1
//table with sorted attribute
\t:1000 res:select count i from t where date=2021.03.07
res

1
x
-
0


#### Query 2: Sorted attribute

In [70]:
//query 2
//table with sorted attribute
\t:1000 res:select avg price from t where sym=`AAPL
res

2360
price  
-------
15.0067


### Querying a table with grouped and sorted attribute
Let's apply the grouped attribute to the `sym` column and sorted attribute on the `date` column:

In [71]:
//table with sorted attribute
t:update `g#sym from t
meta t

c    | t f a
-----| -----
sym  | s   g
date | d   s
price| f    


#### Query 1: Sorted and grouped attribute

In [72]:
//query 1
//table with sorted and grouped attribute
\t:1000 res:select count i from t where date=2021.03.07
res

1
x
-
0


#### Query 2: Sorted and grouped attribute

In [73]:
//query 2
//table with sorted and grouped attribute
\t:1000 res:select avg price from t where sym=`AAPL
res

1001
price  
-------
15.0067


With this small experiment, we can see that attributes improve the performance on table lookups. 

Here is a list of functions that use attributes to improve the performance:

* **qSQL** - When we are querying tables, we will see our code running faster with `where =, where in and where within`
* **Searching**: [bin](https://code.kx.com/q/ref/bin/), [distinct](https://code.kx.com/q/ref/distinct/), [Find](https://code.kx.com/q/ref/find/) and [in](https://code.kx.com/q/ref/in/) 
* **Sorting**: [iasc](https://code.kx.com/q/ref/asc/#iasc) and [idesc](https://code.kx.com/q/ref/asc/##idesc)
* **Dictionaries**: [group](https://code.kx.com/q/ref/group/)

## Partitioned Database- Applying attributes

Attributes that require a particular data structure (<code>\`p#</code> and <code>\`s#</code>) are usually applied at the time of structuring. Other attributes e.g. grouping (<code>\`g#</code>) however, may be applied after data has already been written to disk - this section details how we can use amend (`@`) to apply these directly to on-disk data.

Let's begin by creating a table: 

In [74]:
n:100000
trade:([]timestamp:asc n?.z.P;sym:n?`BAC`BTU`DIS`GE`T;price:n?500f; size:n?100 200 500 1000;ex:n?`NYSE`NASDAQ`LSE`JPX)
meta trade

c        | t f a
---------| -----
timestamp| p   s
sym      | s    
price    | f    
size     | j    
ex       | s    


In [75]:
.Q.dpft[`:db;.z.d;`sym;`trade] //create a hdb for the trade table 

trade


In [76]:
\l db
meta trade

c        | t f a
---------| -----
date     | d    
sym      | s   p
timestamp| p    
price    | f    
size     | j    
ex       | s    


Suppose we wanted to apply a grouped attribute to `ex`, we can use the dbmaint functions that we have seen earlier in the course :  

In [ ]:
setattrcol[`:.;`trade;`ex;`g]

Reloading our database, we will see the attribute has now been applied. 

In [ ]:
\l . 
meta trade    //g attribute on `ex column now 

##### Quiz Time!
Try the Exercises to test your attributes knowledge!